# Lecture 2 - 07.06.2021 

# Exercise: Azimuth compression, part 2

In [9]:
# --- To get started, we set the ipympl backend, which makes matplotlib plots interactive.
# --- We do this using a magic command, starting with %.
%matplotlib widget

# --- import libraries

import sys
sys.path.append('/projects/src/')

import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq, fftshift, ifft

# --- import rat class
from ste_io import *

# --- create useful functions

def rect(t, T0):
    # Implements y = rect(t/T0) = 1 if abs(t/T0) <= 0.5, 0 otherwise 
    # All input / outputs are Numpy arrays
    
    N = t.shape[0]
    y = np.zeros(N, 'float32') 
    y[np.abs(t/T0) <= 0.5] = 1
    
    return y




Consider the following input parameters (DLR's F-SAR platform, P-band, AfriSAR campaign):


In [10]:

# --- Inputs

# speed of light
c0 = 2.9972190e+08
# SAR wavelength
lambda0 = 0.68901587

# --- parameters for range focusing

# range chirp duration, s
tau = 1.0016000e-05
# chirp bandwidth, Hz
Wr  = 50.0e6
# range sampling, m
rsp = 1.1988876
# number of samples 
Nsr = 3772
# range delay, s <---- NEW !!!
rd  = 4.3008000e-05
# the range delay is the amount of time between the chirp transmission
# and the receipt of the first backscattered signal

# --- parameters for azimuth focusing

# azimut resolution, m
deltasa = 2/1.33 # 1.5
# platform velocity, meters/second
vp      = 102.89772    
# pixel spacing in meters
azsp    = 0.90879266    
# number of pixels 
Nsaz    = 9984


Consider now the raw data in ./data/01-sar/raw-img.rat :

In [11]:

img = rrat('./data/01-sar/raw-img.rat')
print(img.shape)

plt.figure()
plt.imshow(abs(img), vmin = 0, vmax = 2*np.mean(np.abs(img)), cmap = 'gray')
plt.title('Raw data')


(9984, 3772)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Raw data')

**Now focus in both range and azimuth !**

In both cases, the matched filters in range and azimuth have to be defined, and applied to the data (*for* loops in both the range and azimuth dimensions).

* Range focusing : 


In [12]:

# generation of the range time axis
Tdr   = (Nsr - 1) * rsp *2 / c0           # total duration
rtime = np.linspace(-Tdr/2, Tdr/2, Nsr)   
# chirp rate in range
kr = Wr / tau

# frequency axis + create hamming ... to make the image nicer !
fs = 1/(rtime[1]-rtime[0])
rfreq = np.linspace(-fs/2, fs/2, Nsr)
iir = np.where(np.abs(rfreq) <= Wr/2 )
nel = iir[0].size
winwin = np.hamming(nel)

# generate chirp
rchirp = np.exp(1j*np.pi*kr*(rtime)**2) * rect(rtime, tau)

# matched filter
matched_filter_r = np.conj(rchirp[::-1])
matched_filter_r_spectrum = fft(matched_filter_r)
matched_filter_r_spectrum[np.abs(matched_filter_r_spectrum) < 1e-6] = 1e2 / Nsr  # to avoid small numbers ...

# focused image in range
imgr = np.zeros((img.shape), 'complex64')
# image with the range chirp
imrchirp = np.zeros((img.shape), 'float32')

for nn in range(img.shape[0]):
    
    imrchirp[nn, :] = np.real(rchirp)
    out = fftshift( fft(img[nn, :]) *  matched_filter_r_spectrum )
    
    # put hamming
    out[iir[0]] = out[iir[0]] * winwin  # apply the window to make the image nicer

    imgr[nn, :] = fftshift(ifft(fftshift(out, -1)))
    
    
plt.figure()
plt.imshow(abs(imgr), vmin = 0, vmax = 2*np.mean(np.abs(imgr)), cmap = 'gray')
plt.title('After range focusing')

plt.figure()
plt.imshow(abs(imrchirp), vmin = 0, vmax = 1, cmap = 'jet')
plt.title('Range chirp')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Range chirp')

* Azimuth focusing :

Hint: adapt the procedure used last time to a range-varying azimuth illumination time !

In [13]:

# focused image in range and azimuth
imgra = np.zeros((img.shape), 'complex64')
# image with the azimuth chirp
imgrachirp = np.zeros((img.shape), 'float32')

# generation of azimuth time axis
ts = azsp / vp                                  # duration of 1 sample
Tdaz = (Nsaz-1) * ts                            # total duration
aztime = np.linspace(-Tdaz/2, Tdaz/2, Nsaz)

# azimuth illumination time
da = deltasa*2                   # real antenna aperture
theta = lambda0 / da             # angular resolution


# frequency axis + create hamming ... to make the image nicer !
azbw = 1 / (deltasa / vp)        # azimuth bandwidth (Hz)
fs = 1/(aztime[1]-aztime[0])
azfreq = np.linspace(-fs/2, fs/2, Nsaz)
iia = np.where(np.abs(azfreq) <= azbw/2 )
nel = iia[0].size
winwin = np.hamming(nel)


for nn in range(imgr.shape[1]):
    
    R0 = rd * c0/2 + nn*rsp
    tsa = theta*R0 / vp              # illumination time (on ground)

    kaz = azbw / tsa                 # azimuth chirp rate

    # Azimuth chirp
    azchirp = np.exp(1j*np.pi*kaz*(aztime)**2)*rect(aztime, tsa)
    
    # matched filter
    matched_filter_a = np.conj(azchirp[::-1])
    matched_filter_a_spectrum = fft(matched_filter_a)
    matched_filter_a_spectrum[np.abs(matched_filter_a_spectrum) < 1e-9] = 1e-9 + 1j*1e-9  # to avoid small numbers ...
    
    # focus !
    imgrachirp[:, nn] = np.real(azchirp)
    out = fftshift( fft(imgr[:, nn]) * matched_filter_a_spectrum )
    
    # put hamming
    out[iia[0]] =  out[iia[0]] * winwin    # apply the window to make the image nicer !
    
    imgra[:, nn] = fftshift(ifft(fftshift(out, -1)))
    

plt.figure()
plt.imshow(abs(imgrachirp), vmin = 0, vmax = 1, cmap = 'jet')
plt.title('Azimuth chirp')

plt.figure(figsize = (5, 8))
plt.imshow(abs(imgra), vmin = 0, vmax = 2.5*np.mean(np.abs(imgra)), cmap = 'gray')
plt.title('After range and azimuth focusing')

plt.figure(figsize = (5, 8))
plt.imshow(np.angle(imgra), cmap = 'jet', interpolation = 'nearest')
plt.title('After range and azimuth focusing')
    



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'After range and azimuth focusing')

68.59848000000001


6034


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …